In [2]:
import os

In [3]:
os.chdir("../")

In [5]:
%pwd

'c:\\MLOps\\CICD\\End-to-End-ML'

In [35]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path:Path

In [36]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories 

In [37]:
class ConfigurationManager:
    def __init__(
       self, 
       config_filepath=CONFIG_FILE_PATH,
       params_filepath=PARAMS_FILE_PATH,
       schema_filepath=SCHEMA_FILE_PATH):     
       
       self.config=read_yaml(config_filepath)
    #    self.params=read_yaml(params_filepath)
       self.schema=read_yaml(schema_filepath)
       create_directories([self.config.artifacts_root])

    def get_transformation_config(self)->DataTransformationConfig:
        config=self.config.data_transformation
        create_directories([config.root_dir])

        # create an object by initialization
        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config          
    


In [38]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [39]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config=config

    def train_test_splitting(self):
        data=pd.read_csv(self.config.data_path)
        data.drop(['Unnamed: 0','RowNumber'], axis=1, inplace=True)
        data.set_index('CustomerId', inplace=True)
        train,test=train_test_split(data)

        train.to_csv( os.path.join(self.config.root_dir,"train.csv"), index=True)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=True)

        logger.info("Splitted data into training and the test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(f"Train Shape :{train.shape}")
        print(f"Test Shape :{test.shape}")


In [41]:
config=ConfigurationManager()
data_transformation_config=config.get_transformation_config()
data_transformation=DataTransformation(config=data_transformation_config)
data_transformation.train_test_splitting()


[2024-03-10 17:28:06,122:INFO:common: yaml file: config\config.yaml loaded successfully]
[2024-03-10 17:28:06,129:INFO:common: yaml file: schema.yaml loaded successfully]
[2024-03-10 17:28:06,129:INFO:common: created directory at: artifacts]
[2024-03-10 17:28:06,137:INFO:common: created directory at: artifacts/data_transformation]
[2024-03-10 17:28:06,226:INFO:3354807181: Splitted data into training and the test sets]
[2024-03-10 17:28:06,226:INFO:3354807181: (7500, 12)]
[2024-03-10 17:28:06,234:INFO:3354807181: (2500, 12)]
Train Shape :(7500, 12)
Test Shape :(2500, 12)
